# 📊 多模态财报 RAG 助手 (ColPali + Qwen-VL)
本 Notebook 演示了如何构建一个能够“看见”财报图表的 RAG 系统：
1. **视觉索引**：使用 ColPali 模型将 PDF 页面直接转换为视觉向量。
2. **多页检索**：针对用户问题，检索 Top-K 张原始页面截图。
3. **智能分析**：将多张截图发送至 Qwen2.5-VL-72B 进行深度财务分析。

### 1. 环境准备
安装 `byaldi` (ColPali 封装库) 及相关依赖。

In [ ]:
pip install byaldi openai tqdm

### 2. 第一阶段：构建视觉索引
使用 ColPali 模型对 PDF 进行索引。ColPali 的优势在于它不需要 OCR，而是直接理解页面的排版和图表。

In [ ]:
import os
from byaldi import RAGMultiModalModel

# 配置环境变量（可选：强制离线或镜像）
os.environ["HF_HUB_OFFLINE"] = "1"
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

MODEL_PATH = "../models/colpali-v1_2-merged"
PDF_PATH = "../data/annual_report_2024_cn.pdf"
INDEX_NAME = "finance_report_2024"

def build_visual_index():
    if not os.path.exists(MODEL_PATH):
        print("❌ 找不到模型文件夹，请确认路径。")
        return

    # 加载模型 (显存不足建议开启 load_in_4bit=True)
    RAG = RAGMultiModalModel.from_pretrained(MODEL_PATH, verbose=1)

    print(f"📖 正在对 {PDF_PATH} 构建视觉索引...")
    RAG.index(
        input_path=PDF_PATH,
        index_name=INDEX_NAME,
        store_collection_with_index=True,
        overwrite=True
    )
    print(f"✅ 索引已保存至: .byaldi/{INDEX_NAME}")

if os.path.exists(PDF_PATH):
    build_visual_index()
else:
    print("❌ 找不到 PDF 文件，请检查路径。")

### 3. 第二阶段：多模态对话与多页增强检索
配置 LLM 客户端，并实现 Top-K 检索逻辑以应对跨页分析。

In [ ]:
from openai import OpenAI
import base64

# --- 配置 ---
API_KEY = "sk-lrdpxzsnhsbckhjrzekbrtccomruhcwzyrlwbroqwojtwtsw"
BASE_URL = "https://api.siliconflow.cn/v1"
MODEL_NAME = "Qwen/Qwen2.5-VL-72B-Instruct"
RETRIEVAL_K = 4 # 检索前4页，防止目录页干扰

client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

# 加载索引
try:
    RAG = RAGMultiModalModel.from_index(INDEX_NAME)
    print("✅ 检索器就绪")
except: 
    print("❌ 请先运行上一阶段构建索引")

def ask_finance_helper(query):
    # 1. 视觉检索
    results = RAG.search(query, k=RETRIEVAL_K)
    
    # 2. 构造多图 Payload
    messages_content = [
        {
            "type": "text", 
            "text": f"你是一个专业CFO。请根据以下 {len(results)} 张截图回答：{query}。请忽略目录页。"
        }
    ]
    
    print(f"🔍 命中页码: {[res.page_num for res in results]}")
    
    for res in results:
        messages_content.append({
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{res.base64}", "detail": "high"}
        })
        
    # 3. 云端推理
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": messages_content}],
        temperature=0.1
    )
    return response.choices[0].message.content

### 4. 运行查询测试

In [ ]:
test_query = "2024年的营收增长率是多少？请结合利润表图表进行说明。"
answer = ask_finance_helper(test_query)
print("\n🤖 CFO助手回答:\n", answer)